In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
import WUBRG
from WUBRG import COLOR_ALIASES_SUPPORT, COLOR_ALIASES, COLOUR_GROUPINGS, COLOUR_GROUPS

import set_info
from set_info import SETS, FORMATS, STAT_NAMES
DATA_CACHE = None
PANDAS_CACHE = None
print(f'SETS:          {str(SETS)}')
print(f'FORMATS:       {str(FORMATS)}')

WRANING: 'discord' failed to import. Emoji support unavailable
SETS:          ['MID']
FORMATS:       ['PremierDraft', 'TradDraft']


In [3]:
def get_top(s, f, c, col, n=5, card_color=None, card_rarity=None):
    frame = PANDAS_CACHE[s][f][c]
    
    if card_color is not None:
        frame = frame[frame['Color'] == card_color]
        
    if card_rarity is not None:
        frame = frame[frame['Rarity'] == card_rarity]
    
    # TODO: Add structure to handle reversing order.
    return frame[col].nlargest(n=n)

In [4]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(df, rarity, iwd_thresh, func1, func2, cols=None):
    by_rarity = df[df['Rarity'] == rarity]
    games_played_mean = by_rarity['# GP'].mean()
    cards = by_rarity[func1(by_rarity['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    if cols is not None:
        cards = cards[cols]
    return cards


def get_trap_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, gt, cols)

def get_trap_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_trap_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_niche_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, lt, cols)

def get_niche_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_niche_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_staple_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, gt, cols)

def get_staple_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_staple_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_dreg_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, lt, cols)

def get_dreg_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_dreg_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)

In [5]:
set_info.init_cache()
DATA_CACHE = set_info.DATA_CACHE
PANDAS_CACHE = set_info.PANDAS_CACHE

Checking for new format data...
MID PremierDraft data is up to date!
MID TradDraft data is up to date!
Done checking for new format data.

Checking for cache updates...
Parsing MID_PremierDraft.json...
MID PremierDraft added to DATA_CACHE!
Parsing MID_TradDraft.json...
MID TradDraft added to DATA_CACHE!
Done checking for cache updates.



In [6]:
premier = PANDAS_CACHE['MID']['PremierDraft']['']
premier['GP WR'].replace(premier['GP WR']*100, inplace=True)
premier_commons = premier[premier['Rarity'] == 'common']
premier_commons['GP WR'].nlargest(n=5)

Series([], Name: GP WR, dtype: float64)

In [7]:
colours = ''
rarity = 'U'
get_trap_cards('MID', 'PremierDraft', colours, rarity, 0.005)

,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD,Color,Rarity
Beloved Beggar,21300,3.650,5224,4.542,21740,55.547,4357,54.120,6768,54.876,11125,54.580,11461,56.191,-1.611,W,U
Odric's Outrider,19035,3.193,4870,3.870,20804,55.018,3847,53.132,6360,54.906,10207,54.237,11403,55.380,-1.143,W,U
Corpse Cobble,30958,5.448,5900,7.382,17110,57.253,2950,53.932,5324,58.302,8274,56.744,9152,57.725,-0.981,UB,U
Firmament Sage,23428,3.980,5537,5.074,20310,53.988,3668,51.200,6978,54.930,10646,53.645,10550,54.483,-0.839,U,U
Covert Cutpurse,19471,3.213,4919,4.070,22547,55.191,4236,53.116,6462,55.850,10698,54.767,12381,55.416,-0.648,B,U
Outland Liberator,20943,3.518,4426,4.602,19709,55.320,4046,55.709,5632,54.439,9678,54.970,10729,55.485,-0.515,G,U
Covetous Castaway,21542,3.727,7676,4.391,35413,57.970,7286,57.851,11281,57.717,18567,57.769,18482,57.970,-0.201,U,U


In [8]:
colours = ''
rarity = 'U'
get_niche_cards('MID', 'PremierDraft', colours, rarity, 0.005)

,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD,Color,Rarity
Sacred Fire,26490,4.637,4497,6.186,10075,53.935,2277,57.444,3393,58.591,5670,58.131,5271,50.655,7.476,WR,U
Delver of Secrets,26070,4.707,3637,6.773,8906,54.592,1852,62.257,3204,55.493,5056,57.971,4465,51.467,6.504,U,U
Vampire Socialite,23622,4.261,4278,5.435,12060,55.174,2781,62.316,3845,55.345,6626,58.270,6302,52.682,5.589,BR,U
Sunrise Cavalier,23927,4.192,3939,5.537,9623,53.902,2187,59.808,3255,54.747,5442,56.781,4885,51.259,5.522,WR,U
Kessig Naturalist,22918,4.402,3257,5.957,7918,50.871,1757,56.118,2476,51.939,4233,53.674,4171,48.166,5.508,RG,U
Rootcoil Creeper,29168,5.324,6099,6.624,13766,55.840,3333,61.266,5409,56.424,8742,58.270,6395,53.057,5.213,UG,U
Turn the Earth,37731,7.364,4452,10.916,2061,52.790,350,50.571,625,57.760,975,55.179,1141,51.271,3.909,G,U
Dawnhart Wardens,27116,4.771,4675,6.317,13169,57.240,2906,60.014,4465,57.962,7371,58.771,6691,55.657,3.114,WG,U
Winterthorn Blessing,33876,6.172,5473,8.482,9485,55.983,1927,54.593,3394,58.898,5321,57.339,4797,54.909,2.430,UG,U
Seize the Storm,30168,5.303,5222,6.918,11615,51.847,2352,48.002,4959,56.221,7311,53.577,5327,51.211,2.366,R,U


In [9]:
colours = ''
rarity = 'U'
get_staple_cards('MID', 'PremierDraft', colours, rarity, 0.005)

,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD,Color,Rarity
Bladestitched Skaab,22515,3.903,7018,4.460,27121,59.367,5908,63.236,11177,63.255,17085,63.248,12740,55.722,7.526,UB,U
Morbid Opportunist,13191,2.269,7227,2.438,36163,58.292,7133,62.666,12404,61.730,19537,62.072,18872,54.970,7.102,B,U
Fleshtaker,26220,4.535,5982,5.631,17240,56.839,3801,60.405,6565,60.427,10366,60.419,8539,53.671,6.747,WB,U
Rise of the Ants,24269,4.068,6115,4.950,23491,57.243,4222,59.356,7550,61.669,11772,60.839,12713,54.566,6.273,G,U
Ambitious Farmhand,16029,2.742,5864,3.153,28045,58.723,5735,62.459,9092,61.736,14827,62.015,14774,55.970,6.045,W,U
Skaab Wrangler,26302,4.541,9562,5.049,39799,59.708,8391,61.340,15211,63.336,23602,62.626,19186,56.672,5.955,U,U
Dreadhound,17018,2.886,6439,3.322,30711,56.963,5304,58.428,10552,60.974,15856,60.122,16197,54.424,5.699,B,U
Overwhelmed Archivist,20015,3.400,7575,3.900,36544,58.830,7281,62.546,11996,61.404,19277,61.835,18918,56.343,5.492,U,U
Foul Play,18164,2.970,6490,3.460,31530,58.316,6099,59.780,9510,62.292,15609,61.311,16952,55.893,5.418,B,U
Deathbonnet Sprout,22937,3.937,5127,4.978,19229,56.280,4043,62.701,5961,56.836,10004,59.206,10158,53.997,5.209,G,U


In [10]:
colours = ''
rarity = 'U'
get_dreg_cards('MID', 'PremierDraft', colours, rarity, -0.005)

,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD,Color,Rarity
Field of Ruin,38897,7.521,4378,11.234,768,48.438,122,42.623,209,44.498,331,43.807,441,51.701,-7.894,,U
Necrosynthesis,30729,5.512,2966,8.960,3279,48.917,583,42.882,886,46.727,1469,45.201,1829,51.777,-6.576,B,U
Unnatural Moonrise,31018,5.704,3448,8.560,5583,49.382,1078,46.197,1587,45.999,2665,46.079,3123,51.585,-5.506,RG,U
Village Watch,23774,4.213,3159,5.857,9054,50.320,1466,45.293,2586,48.608,4052,47.409,5241,52.223,-4.814,R,U
Ghoulish Procession,21415,3.626,3749,4.784,13876,52.883,2565,50.448,4025,50.634,6590,50.561,7525,54.777,-4.216,B,U
Heirloom Mirror,25695,4.466,4140,6.109,11835,53.046,2244,51.203,3237,50.880,5481,51.013,6557,54.659,-3.647,B,U
Bereaved Survivor,21445,3.664,3624,4.999,13877,53.614,2557,50.254,4188,52.746,6745,51.801,7578,55.173,-3.372,W,U
Cathar's Call,25681,4.564,2568,7.118,5675,52.828,1027,50.730,1613,51.085,2640,50.947,3151,54.173,-3.226,W,U
Vivisection,32294,5.711,5585,8.024,13567,54.235,2296,51.524,4091,53.361,6387,52.701,7392,55.628,-2.927,U,U
Mysterious Tome,27144,4.757,5237,6.328,16204,53.894,2924,50.137,4979,54.168,7903,52.676,8542,55.081,-2.405,U,U


In [11]:
colours = ''
rarity = 'U'
get_top('MID', 'PremierDraft', colours, 'IWD', card_color='U', card_rarity=rarity)
get_top('MID', 'PremierDraft', colours, 'IWD', card_color='B', card_rarity=rarity)
get_top('MID', 'PremierDraft', colours, 'IWD', card_color='U', card_rarity=rarity)

Delver of Secrets        6.504
Skaab Wrangler           5.955
Overwhelmed Archivist    5.492
Nebelgast Intruder       4.738
Fading Hope              3.580
Name: IWD, dtype: float64

In [12]:
get_top('MID', 'PremierDraft', colours, 'IWD', 10, card_color='B', card_rarity=rarity)

Morbid Opportunist      7.102
Dreadhound              5.699
Foul Play               5.418
Infernal Grasp          5.023
Vengeful Strangler      2.439
Bloodtithe Collector    0.968
Baneblade Scoundrel     0.773
Stromkirk Bloodthief    0.135
Covert Cutpurse        -0.648
Heirloom Mirror        -3.647
Name: IWD, dtype: float64

In [13]:
x = get_top('MID', 'PremierDraft', colours, 'IWD', card_color='B', card_rarity=rarity)
print(str(x))

Morbid Opportunist    7.102
Dreadhound            5.699
Foul Play             5.418
Infernal Grasp        5.023
Vengeful Strangler    2.439
Name: IWD, dtype: float64
